In [ ]:
## Topic Modelling of DL Definitions

In [1]:
import pandas as pd
from gensim import corpora, models, similarities
import bleach
from nltk.corpus import stopwords
from nltk import ConditionalFreqDist, FreqDist
import nltk
import gexf as g
import collections
import os.path

# Fix invalid display error
%matplotlib inline


In [2]:
indirname = "/home/paulus/docs/datalit/"
paperlist = []
paperdict = dict()
   
for fn in os.listdir(indirname ):
    if fn.endswith(".txt"):
        print "opening - " + fn
        hdl = open(indirname + fn)
        txt = hdl.read()
        paperlist.append(txt)
        
        index_of_dot = fn.index('.')
        fns = fn[:index_of_dot]
        paperdict[fns]=dict()
        paperdict[fns]["text"] = txt


opening - Carlson et al. - 2011 - Determining Data Information Literacy Needs A Study of Students and Research Faculty.pdf.txt
opening - Fonticiaro, Oehrli - 2016 - Why data literacy matters.pdf.txt
opening - blog - sod - defining dl.txt
opening - Teal et al. - 2015 - Data Carpentry Workshops to Increase Data Literacy for Researchers.pdf.txt
opening - Determining Data Information Literacy Needs- A Study of Students.pdf.txt
opening - Erwin - 2015 - Data Literacy Real-World Learning Through Problem-Solving With Data Sets.pdf.txt
opening - Data-Pop Alliance - 2015 - Beyond Data Literacy Reinventing Community Engagement and Empowerment in the Age of Data.pdf.txt
opening - blog- Advancing Data Literacy.txt
opening - Martin - 2014 - What Is Data Literacy.pdf.txt
opening - Mandinach, Gummer - 2013 - A Systemic View of Implementing Data Literacy in Educator Preparation.pdf.txt
opening - Koltay - 2015 - Data literacy in search of a name and identity.pdf.txt
opening - Anderson et al. - 2015 - Et

In [ ]:
# remove common words and tokenize
stoplist = stopwords.words('english')
stoplist.extend([u'',u'data',u'literacy',u'data,', u'dil', u'training',
                 u'information',u'research',u'faculty',u'big',u'used',u'work',
                 u'researchers', u'student',u'education',u'educators',u'skills',
                 u'librarians',u'learning',u'ability', u'understanding',
                 u'management',u'using',u'knowledge',u'know', u'course',u'social',u'different',
                 u'also',u'al.',u'al.,',
                 u'et',u'no.',u'literacy.',u'students',u'data.',u'june',u'association\nmay',
                 u'literacy',u'michigan',u'literacy,',u'de',u'data-related',
                 u"don't",u"isn't",u'want',u'get',u'best',u'things',u'‘Hi’',u'it’s',u"don’t",
                 u'even',u"we're",u'like',u'&',u'may',u'must', u'j.',u'e.',u'data-driven',
                 u'use', u'nih',u'j,', u'need', u'new',u'needs', u'acrl',
                 u'would',u'one',u'think',u'might',u'could',u'it\'s',u'us',u'don\'t',u'i\'m'])
texts = [[word for word in paper.lower().split(" ") if word not in stoplist]
for paper in paperlist]

# remove words that appear only once
all_tokens = sum(texts, [])
gensim.utils.lemmatize(document, allowed_tags=re.compile('(NN|VB|JJ|RB)'))
tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) == 1)
texts = [[word for word in text if word not in tokens_once]
         for text in texts]



In [ ]:
dictionary = corpora.Dictionary(texts)
dictionary.save('../output/dl.dict') # store the dictionary, for future reference
print(dictionary)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('../output/dl.mm', corpus) # store to disk, for later use

In [ ]:
#lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=8, onepass=False)
#lsi.print_topics(200)

lda = models.LdaModel(corpus, id2word=dictionary,num_topics=6, passes=5,  iterations=5)
lda.print_topics(num_topics=3, num_words=6)

In [3]:
def is_useful(tag):
    return tag.lower() in ['nn','nns','np','nnp','vb','vbj','vbn','vbg','vbd','vbp','vbz'] 

sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
stoplist = stopwords.words('english')
theData = collections.Counter()

for name in paperdict.keys():
    try:
        paperdict[name]["content"] = collections.Counter()
        paper = paperdict[name]["text"].decode('utf-8')
        sentences = sent_tokenizer.tokenize(paper)
        for sentence in sentences:
            content =  nltk.word_tokenize(sentence)
            tagged_content = nltk.pos_tag(content)

            for (index, tagtuple) in enumerate(tagged_content):
                    (token, tag) = tagtuple
                    token = token.lower()
                    if token not in stoplist and is_useful(tag) and len(token)>2:
                            theData[token] +=1
                            paperdict[name]["content"][token]+=1
    except UnicodeDecodeError as e:
            print "paper didn't work!"


print "OK Fred!!"
                

OK Fred!!


In [4]:
print theData.most_common(150)

[(u'data', 2474), (u'literacy', 760), (u'information', 440), (u'students', 428), (u'research', 312), (u'use', 233), (u'skills', 198), (u'need', 190), (u'researchers', 179), (u'faculty', 161), (u'student', 139), (u'education', 139), (u'management', 138), (u'analysis', 127), (u'work', 118), (u'needs', 116), (u'knowledge', 113), (u'training', 109), (u'course', 103), (u'using', 100), (u'understanding', 98), (u'librarians', 93), (u'learning', 91), (u'know', 90), (u'development', 89), (u'standards', 80), (u'used', 80), (u'tools', 78), (u'ability', 77), (u'metadata', 76), (u'understand', 76), (u'science', 73), (u'practices', 73), (u'project', 70), (u'people', 68), (u'practice', 68), (u'program', 65), (u'community', 65), (u'needed', 64), (u'make', 63), (u'time', 62), (u'making', 62), (u'set', 61), (u'part', 60), (u'libraries', 60), (u'educators', 59), (u'university', 59), (u'quality', 58), (u'provide', 57), (u'media', 57), (u'access', 57), (u'programs', 56), (u'dil', 56), (u'sharing', 56), (u'

In [10]:
print paperdict[paperdict.keys()[5]]['content'].most_common(150)


[(u'data', 155), (u'education', 61), (u'use', 52), (u'literacy', 52), (u'educators', 46), (u'schools', 41), (u'courses', 30), (u'al.', 29), (u'making', 27), (u'need', 27), (u'decision', 26), (u'development', 24), (u'mandinach', 23), (u'knowledge', 21), (u'systems', 20), (u'needs', 17), (u'practice', 17), (u'preparation', 17), (u'skills', 16), (u'teachers', 15), (u'organizations', 14), (u'policy', 14), (u'research', 14), (u'student', 14), (u'training', 13), (u'states', 13), (u'districts', 12), (u'decisions', 11), (u'standards', 11), (u'address', 11), (u'teacher', 11), (u'change', 11), (u'field', 11), (u'educator', 10), (u'state', 10), (u'practices', 10), (u'evidence', 10), (u'requirements', 9), (u'capacity', 9), (u'agencies', 9), (u'learning', 9), (u'understand', 9), (u'stakeholders', 9), (u'using', 9), (u'gummer', 9), (u'preservice', 9), (u'accreditation', 8), (u'love', 8), (u'inform', 8), (u'faculty', 8), (u'system', 8), (u'needed', 8), (u'programs', 8), (u'make', 8), (u'components', 